# twitter sentiment analysis for classification of racist/sexist tweets 

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
# nltk.download(stopwords)
from textblob import TextBlob

In [70]:
import re

In [35]:
from tqdm import tqdm

In [2]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [3]:
from nltk.stem import PorterStemmer
st = PorterStemmer()

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [27]:
df = pd.read_csv("twitter_analysis/train.csv")
# df.head()

In [6]:
df.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [11]:
# df['new_tweet'] = np.nan
# df = df.fillna('')

### only alphabets and hashtag (remove punctuation, numbers and special char)

In [28]:
df.new_tweet = df.tweet.str.replace('[^a-zA-Z#]',' ')
df.new_tweet.head()

0      user when a father is dysfunctional and is s...
1     user  user thanks for #lyft credit i can t us...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide  society now    #motivation
Name: tweet, dtype: object

### Remove word user and also words with length less than 3

In [29]:
df.new_tweet=df.new_tweet.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df.new_tweet=df.new_tweet.apply(lambda x: ' '.join([w for w in x.split() if w != "user"]))

In [30]:
df.new_tweet.head()

0    when father dysfunctional selfish drags kids i...
1    thanks #lyft credit cause they offer wheelchai...
2                                  bihday your majesty
3                           #model love take with time
4                       factsguide society #motivation
Name: tweet, dtype: object

In [37]:
# for i in tqdm(range(len(df))):
#     filtered_sent=[]
#     for x in df.new_tweet[i].split():
#         x = str(TextBlob(x).correct())
#         x = st.stem(x)
#         if x.lower() not in stop:
#             filtered_sent.append(x.lower())
#     df.new_tweet[i] = " ".join(filtered_sent)


In [38]:
df_new = df.new_tweet.apply(lambda x: x.split())

### Spelling Correction with stemmingg and conert words to lower case

In [41]:
df_new = df_new.apply(lambda x: [st.stem(str(TextBlob(i).correct())).lower() for i in x if i.lower() not in stop])
# df_new = df_new.apply(lambda x: [st.stem(i) for i in tqdm(x)])
# df_new = df_new.apply(lambda a: [x.lower() for x in tqdm(a) if x.lower() not in stop])

In [42]:
# df_new.to_csv('processed_tweets.csv')

In [43]:
df_new = df_new.apply(lambda y: ' '.join(y))

In [56]:
# df_new.to_csv('processed_tweets.csv')

## New Column 'processed_tweets' contains preprocessed tweets 

In [57]:
df['processed_tweets']=df_new
df.head()

,id,label,tweet,processed_tweets
0,1,0,@user when a father is dysfunctional and is s...,father dysfunct selfish drag kiss dysfunct #run
1,2,0,@user @user thanks for #lyft credit i can't us...,thank #left credit caus offer wheelchair van #...
2,3,0,bihday your majesty,midday majesti
3,4,0,#model i love u take with u all the time in ...,#model love take time
4,5,0,factsguide: society now #motivation,factsguid societi #motiv


In [58]:
df.to_csv('twitter_analysis/processed_tweet.csv')

In [59]:
df.isnull().sum()

id                  0
label               0
tweet               0
processed_tweets    0
dtype: int64

In [ ]:
df['tweet'].apply(lambda x: x.split()).head()

In [61]:
positive_sent = []
negative_sent = []
for i in tqdm(range(len(df))):
    if df['label'][i]==1:
        for x in df['processed_tweets'][i].split():
            negative_sent.append(x)
    else:
        for x in df.processed_tweets[i].split():
            positive_sent.append(x)  


100%|█████████████████████████████████| 31962/31962 [00:02<00:00, 13878.42it/s]

In [62]:
# M1 --> for i in set(negative_sent):
#     negative_sent.count(i)
positive_sentences = " ".join(positive_sent)
negative_sentences = " ".join(negative_sent)

### Most counts words in non racist/sexist tweets

In [66]:
top_pos_freq = pd.Series(positive_sentences.split()).value_counts()[:20]
top_pos_freq

happi     1759
#love     1716
love      1544
time      1191
today     1016
thank     1005
like       982
make       954
#posit     835
good       796
want       756
peopl      749
father     741
life       740
take       728
look       716
#smile     674
come       673
day        659
feel       648
dtype: int64

### Most count words in racist/sexist tweets

In [64]:
top_neg_freq = pd.Series(negative_sentences.split()).value_counts()[:20]
top_neg_freq

racism        173
like          145
white         143
#tramp        136
black         103
#polit         95
#allahsoil     92
stop           88
peopl          84
tramp          77
might          77
#librari       76
#saw           75
#liber         75
librari        73
women          68
call           64
#between       63
feel           63
listen         61
dtype: int64

In [115]:
# FUNCTION TO EXTRACT HASHTAG WORDS
def hashtag(x):
    hashtags = []
    for tweet in x:
#         if re.search(r'#\w+', tweet)!= None:
        ht=re.findall(r"#(\w+)", tweet)
        hashtags.append(ht)
    return hashtags

#### HASHTAG word with counts in racist/sexist tweeets 

In [116]:
negative_hashtag=hashtag(df['processed_tweets'][df['label']==1])
negative_hashtag = sum(negative_hashtag, [])
# negative_hashtag[0:30]

In [117]:
top_neg_ht_words = pd.Series(negative_hashtag).value_counts()[:20]
top_neg_ht_words

tramp        136
polit         95
allahsoil     92
librari       76
saw           75
liber         75
between       63
mimi          46
black         46
hate          37
right         33
drama         32
camp          32
ulm           28
breast        27
salari        27
sigh          27
tempt         26
templ         26
hispan        26
dtype: int64

#### HASHTAG word with counts in non racist/sexist tweeets 

In [120]:
positive_hashtag=hashtag(df['processed_tweets'][df['label']==0])
positive_hashtag = sum(positive_hashtag, [])
top_pos_ht_words = pd.Series(positive_hashtag).value_counts()[:20]
top_pos_ht_words

love          1765
posit          837
smile          692
thank          539
healthi        529
fun            464
life           452
happi          436
affirm         423
summer         399
beauti         387
model          377
cut            371
day            368
blow           352
follow         348
friend         346
fathersday     341
girl           311
silver         304
dtype: int64

In [125]:
x_train, x_test, y_train, y_test = train_test_split(df["processed_tweets"],df["label"], test_size = 0.2, random_state = 3)

In [124]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [128]:
# len(x_test)

6393

# Tfidf count words present in each and every tweets

In [133]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['processed_tweets'])
xtrain_tfidf =  tfidf_vect.transform(x_train)
xtest_tfidf =  tfidf_vect.transform(x_test)

# Random forest model to features defined by tfidf

In [138]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(xtrain_tfidf,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [139]:
prediction = model.predict(xtest_tfidf)
prediction[0:5]

array([0, 0, 0, 0, 1], dtype=int64)

In [140]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,prediction))

0.963084623807


##### Accuracy: 96%  on validation data

In [141]:
test = pd.read_csv('twitter_analysis/test.csv')
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [142]:
test.new_tweet = test.tweet.str.replace('[^a-zA-Z#]',' ')
test.new_tweet=test.new_tweet.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
test.new_tweet=test.new_tweet.apply(lambda x: ' '.join([w for w in x.split() if w != "user"]))

In [143]:
test_new = test.new_tweet.apply(lambda x: x.split())
test_new = test_new.apply(lambda x: [st.stem(str(TextBlob(i).correct())).lower() for i in x if i.lower() not in stop])
processed_test = test_new.apply(lambda y: ' '.join(y))

In [144]:
processed_test.head()

0    #studiolif #dislik #requir #passion #educ #wil...
1         #white #supremacist want everyon #bird #movi
2        safe way heal #acn #altwaystoh #healthi #heal
3    curs child book reserv alreadi #harrypott #pot...
4    #midday amaz hilari #nephew their uncl dave lo...
Name: tweet, dtype: object

In [145]:
test['processed_tweets'] = processed_test 

In [150]:
test.to_csv('twitter_analysis/processed_test.csv')

In [147]:
test_tfidf =  tfidf_vect.transform(test['processed_tweets'])
predict_test = model.predict(test_tfidf)

##### Accuracy: 71% on test data 

In [149]:
final_result = pd.DataFrame({'id':test['id'],'label':predict_test})
final_result.to_csv('output.csv',index=False)